In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import sys
import os
from pprint import pprint
import IPython.display as ipd ## Used to allow us to listen to audio in notebook
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./UrbanSound8K.csv') 

In [3]:
pip install librosa

Note: you may need to restart the kernel to use updated packages.


In [5]:
import librosa
import librosa.display

In [6]:
df.count()

slice_file_name    8732
fsID               8732
start              8732
end                8732
salience           8732
fold               8732
classID            8732
class              8732
dtype: int64

In [7]:
y, sr = librosa.load('./fold1/7061-6-0-0.wav')
sr

22050

In [8]:
import librosa
import numpy as np

arr, _ = librosa.load('./fold1/7061-6-0-0.wav', res_type="kaiser_fast")
print(arr.shape)

arr = arr.reshape(-1, 1)
print(arr.shape)

X = []
X.append(arr)
X = np.array(X)
print(X.shape)

(49613,)
(49613, 1)
(1, 49613, 1)


In [9]:
X = []
labels = []

def pad(arr, size):
    """
    Pad the arr to the given size.
    """
    return librosa.util.fix_length(arr, size=size)

for i in range(8732):
    file_name = './fold' + str(df["fold"][i]) + '/' + df["slice_file_name"][i]
    y, sr = librosa.load(file_name, res_type='kaiser_fast', sr=22050)
    y = pad(y, 22050)
    y = y.reshape(-1, 1)
    X.append(y)
    labels.append(df["classID"][i])

In [10]:
X_array = np.array(X)
y_array = np.array(labels)

In [11]:
X_array.shape

(8732, 22050, 1)

In [12]:
y_array.shape

(8732,)

In [18]:
import h5py
hf = h5py.File('data1.h5', 'w')
hf.create_dataset('dataset_1', data=X_array)
hf.create_dataset('dataset_2', data=y_array)
hf.close()

In [11]:
import h5py
hf = hf = h5py.File('data1.h5', 'r')
load_x = hf.get('dataset_1')
load_y = hf.get('dataset_2')

X_array = np.array(load_x)
y_array = np.array(load_y)

In [12]:
X_array.shape

(8732, 22050, 1)

In [13]:
y_array.shape

(8732,)

In [19]:
from tensorflow.keras.utils import to_categorical 
from sklearn.preprocessing import LabelEncoder

labelencoder = LabelEncoder()
y_encoded = to_categorical(labelencoder.fit_transform(y_array))
print(y_array)
print(y_encoded)

[3 2 2 ... 1 1 1]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [20]:
y_encoded.shape

(8732, 10)

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_array, y_encoded, test_size=0.2, random_state=42)

In [22]:
X_array[0].shape

(22050, 1)

In [23]:
model_input=(22050, 1)

In [24]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, LSTM, SimpleRNN, Conv2D, Conv1D, MaxPool1D, MaxPooling2D, Dropout, Flatten

In [28]:
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(model_input)))
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_2 (Conv1D)            (None, 22048, 32)         128       
_________________________________________________________________
flatten_1 (Flatten)          (None, 705536)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                35276850  
_________________________________________________________________
dense_3 (Dense)              (None, 10)                510       
Total params: 35,277,488
Trainable params: 35,277,488
Non-trainable params: 0
_________________________________________________________________


In [31]:
X_train.shape

(6985, 22050, 1)

In [32]:
y_train.shape

(6985, 10)

In [30]:
model.fit(X_train, y_train, epochs = 100, batch_size = 64, validation_data = (X_test, y_test))
predictions = model.predict(X_test)

Epoch 1/100
110/110 [==============================] - 2s 15ms/step - loss: 43.2938 - accuracy: 0.1140 - val_loss: 614.2405 - val_accuracy: 0.1231
Epoch 2/100
110/110 [==============================] - 2s 15ms/step - loss: 5.1737 - accuracy: 0.1187 - val_loss: 2.3743 - val_accuracy: 0.1048
Epoch 3/100
110/110 [==============================] - 2s 16ms/step - loss: 2.3327 - accuracy: 0.1094 - val_loss: 2.3627 - val_accuracy: 0.1311
Epoch 4/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2870 - accuracy: 0.1099 - val_loss: 2.3576 - val_accuracy: 0.1048
Epoch 5/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2814 - accuracy: 0.1104 - val_loss: 2.3413 - val_accuracy: 0.1093
Epoch 6/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2794 - accuracy: 0.1162 - val_loss: 2.3299 - val_accuracy: 0.1311
Epoch 7/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2656 - accuracy: 0.1111 - val_loss: 2.3257 - val_accura

110/110 [==============================] - 2s 14ms/step - loss: 2.2604 - accuracy: 0.1092 - val_loss: 2.2664 - val_accuracy: 0.1162
Epoch 58/100
110/110 [==============================] - 2s 14ms/step - loss: 2.2613 - accuracy: 0.1221 - val_loss: 2.2659 - val_accuracy: 0.1048
Epoch 59/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2635 - accuracy: 0.1160 - val_loss: 2.2757 - val_accuracy: 0.1162
Epoch 60/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2611 - accuracy: 0.1091 - val_loss: 2.2687 - val_accuracy: 0.1048
Epoch 61/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2626 - accuracy: 0.1144 - val_loss: 2.2705 - val_accuracy: 0.1048
Epoch 62/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2639 - accuracy: 0.1142 - val_loss: 2.2691 - val_accuracy: 0.1168
Epoch 63/100
110/110 [==============================] - 2s 15ms/step - loss: 2.2605 - accuracy: 0.1142 - val_loss: 2.2659 - val_accuracy: 0.117

In [33]:
model2 = Sequential()

model2.add(Dense(1024, input_shape=(model_input), activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.2))

model2.add(Flatten()) 
model2.add(Dense(10, activation="softmax"))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [34]:
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 22050, 1024)       2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 22050, 1024)       0         
_________________________________________________________________
dense_5 (Dense)              (None, 22050, 512)        524800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 22050, 512)        0         
_________________________________________________________________
dense_6 (Dense)              (None, 22050, 256)        131328    
_________________________________________________________________
dropout_3 (Dropout)          (None, 22050, 256)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 5644800)          

In [35]:
model2.fit(X_train, y_train, epochs = 100, batch_size = 64, validation_data = (X_test, y_test))
predictions = model.predict(X_test)

Epoch 1/100


2022-04-17 19:41:36.433989: W tensorflow/core/common_runtime/bfc_allocator.cc:434] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.69GiB (rounded to 2890137600)
Current allocation summary follows.
2022-04-17 19:41:36.434010: I tensorflow/core/common_runtime/bfc_allocator.cc:934] BFCAllocator dump for GPU_0_bfc
2022-04-17 19:41:36.434018: I tensorflow/core/common_runtime/bfc_allocator.cc:941] Bin (256): 	Total Chunks: 66, Chunks in use: 65. 16.5KiB allocated for chunks. 16.2KiB in use in bin. 1.9KiB client-requested in use in bin.
2022-04-17 19:41:36.434023: I tensorflow/core/common_runtime/bfc_allocator.cc:941] Bin (512): 	Total Chunks: 5, Chunks in use: 5. 2.5KiB allocated for chunks. 2.5KiB in use in bin. 2.0KiB client-requested in use in bin.
2022-04-17 19:41:36.434027: I tensorflow/core/common_runtime/bfc_allocator.cc:941] Bin (1024): 	Total Chunks: 4, Chunks in use: 4. 4.2KiB allocated for chunks. 4.2KiB in use in bin. 4.0KiB client-requested in use in bin.
2022-04-17

ResourceExhaustedError:  OOM when allocating tensor with shape[64,22050,512] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_2/dropout_2/dropout/random_uniform/RandomUniform (defined at tmp/ipykernel_647125/3355199412.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_71088]

Function call stack:
train_function
